In [27]:
#import dependencies
import pandas as pd
import yfinance as yf
import datetime

### Question 1: [Index] S&P 500 Stocks Added to the Index

**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each company was added to the index.

Hint: you can use [pandas.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) to scrape the data into a DataFrame.

Steps:
1. Create a DataFrame with company tickers, names, and the year they were added.
2. Extract the year from the addition date and calculate the number of stocks added each year.
3. Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).

*Context*: 
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data = pd.read_html(url)

In [12]:
df = data[0]

In [ ]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [13]:
df.columns

Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

In [15]:
df['Date first added'] = pd.to_datetime(df['Date added'], errors='coerce')
df['Year added'] = df['Date first added'].dt.year
year_counts = df['Year added'].value_counts().sort_index()

In [21]:
year_counts = year_counts[year_counts.index != 1957]
most_additions_year = year_counts[year_counts == year_counts.max()].index[-1] 

In [22]:
most_additions_year

2017

_How many current S&P 500 stocks have been in the index for more than 20 years?_

In [25]:
current_year = datetime.datetime.now().year
df['Years in index'] = current_year - df['Year added']
long_tenure_count = (df['Years in index'] > 20).sum()

print("Number of S&P 500 companies with >20 years in index:", long_tenure_count)


Number of S&P 500 companies with >20 years in index: 219


### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)	
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context: 
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.

In [34]:
index_tickers = {
    "United States - S&P 500": "^GSPC",
    "China - Shanghai Composite": "000001.SS",
    "Hong Kong - Hang Seng": "^HSI",
    "Australia - ASX 200": "^AXJO",
    "India - Nifty 50": "^NSEI",
    "Canada - TSX": "^GSPTSE",
    "Germany - DAX": "^GDAXI",
    "UK - FTSE 100": "^FTSE",
    "Japan - Nikkei 225": "^N225",
    "Mexico - IPC": "^MXX",
    "Brazil - Ibovespa": "^BVSP"
}

In [60]:
#analyze returns
results = {index: {} for index in index_tickers.keys()}

periods = {
        'This year': ('2025-01-01', '2025-05-01'),
        '3 Year': ('2022-05-01', '2025-05-01'),
        '5 Year': ('2020-05-01', '2025-05-01'),
        '10 Year': ('2015-05-01', '2025-05-01')
    }

for years, (start_date, end_date) in periods.items():
    for names, symbols in index_tickers.items():
        ticker = yf.Ticker(symbols)
        data = ticker.history(start=start_date, end=end_date)
        
        if not data.empty:
            start_price = data.iloc[0]['Close']
            end_price = data.iloc[-1]['Close']
            ytd_return = ((end_price - start_price) / start_price) * 100
            results[names][years] = ytd_return
        else:
            print(f"{names:<30}: No data available")

In [61]:
import pandas as pd

returns_df = pd.DataFrame(results).T
returns_df.columns.name = "Period"
returns_df.index.name = "Index"
print(returns_df)

Period                      This year     3 Year      5 Year     10 Year
Index                                                                   
United States - S&P 500     -5.103301  34.020480   96.737219  164.150565
China - Shanghai Composite   0.504817   6.886816   13.928827  -26.814921
Hong Kong - Hang Seng       12.720018   4.821935   -6.328463  -21.349909
Australia - ASX 200         -0.914500  10.605692   54.905743   39.759912
India - Nifty 50             2.490424  42.562875  161.841063  192.058866
Canada - TSX                -0.226126  20.053451   69.912379   61.942786
Germany - DAX               12.346378  61.395129  114.936570   93.608190
UK - FTSE 100                2.842590  12.347091   47.401576   21.598918
Japan - Nikkei 225          -8.297931  34.404756   83.723618   84.548741
Mexico - IPC                13.049444   8.425565   54.684126   24.361595
Brazil - Ibovespa           12.438710  26.658164   71.239667  135.497088


In [63]:
import numpy as np

for period in periods.keys():
    period_results = {index: returns_df.loc[index, period] for index in returns_df.index}

    sp500_period = period_results.get('United States - S&P 500', np.nan)

    if not np.isnan(sp500_period):
        better_than_sp500 = sum(
            1 for name, ret in period_results.items()
            if not np.isnan(ret) and ret > sp500_period and name != 'United States - S&P 500'
        )
        print(f"{period}: {better_than_sp500} indices beat the S&P 500 ({sp500_period:.2f}%)")
    else:
        print(f"{period}: S&P 500 return not available.")

This year: 9 indices beat the S&P 500 (-5.10%)
3 Year: 3 indices beat the S&P 500 (34.02%)
5 Year: 2 indices beat the S&P 500 (96.74%)
10 Year: 1 indices beat the S&P 500 (164.15%)


### Question 3. [Index] S&P 500 Market Corrections Analysis


**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:* 
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

### Question 4.  [Stocks] Earnings Surprise Analysis for Amazon (AMZN)


**Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.**

Steps:
1. Load earnings data from CSV ([ha1_Amazon.csv](ha1_Amazon.csv)) containing earnings dates, EPS estimates, and actual EPS
2. Download complete historical price data using yfinance
3. Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the return as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
4. Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
5. Calculate 2-day percentage changes following positive earnings surprises
6. Compare the median 2-day percentage change for positive surprises vs. all historical dates

Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

*Additional*: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)


### Question 5.  [Exploratory, optional] Brainstorm potential idea for your capstone project

**Free text answer**

Describe the capstone project you would like to pursue, considering your aspirations, ML model predictions, and prior knowledge. Even if you are unsure at this stage, try to generate an idea you would like to explore-such as a specific asset class, country, industry vertical, or investment strategy. Be as specific as possible.

*Example: I want to build a short-term prediction model for the US/India/Brazil stock markets, focusing on the largest stocks over a 30-day investment horizon. I plan to use RSI and MACD technical indicators and news coverage data to generate predictions.*

### Question 6. [Exploratory, optional] Investigate new metrics

**Free text answer**

Using the data sources we have covered (or any others you find relevant), download and explore a few additional metrics or time series that could be valuable for your project. Briefly explain why you think each metric is useful. This does not need to be a comprehensive list-focus on demonstrating your ability to generate data requests based on your project description, identify and locate the necessary data, and explain how you would retrieve it using Python.
